In [1]:
import pandas as pd

In [1]:
# First, import the module corresponding to the desired census product.
# For example, let's import the ACS 1-year detailed tables.
import census_products.acs.acs1 as acs1
# NOTE: don't be afraid to use autocomplete to help find available census products!

# Individual datasets within a product can be selected by indexing the imported module by year, like so:
dataset = acs1[2022]
# NOTE: some datasets (like time series datasets) don't have a year.
# Index these using the None key, or calling the module without arguments. (e.g., acs1[None], acs1())

In [ ]:
# Alternatively, you can import the desired dataset directly from the module.
from census_products.acs.acs1 import AcsAcs1_2022 as dataset

# The docstring of each dataset contains some metadata about it.
# If the dataset was imported directly from a module, most IDEs should be able to show it directly.
# Try hovering your mouse over the following statement:
dataset

# Alternatively, each dataset has properties corresponding to its metadata.
display(f"Title: {dataset.title}")
display(f"Description: {dataset.description}")
display(f"Vintage: {dataset.vintage}")
display(f"API Endpoint: {dataset.api_endpoint}")

In [ ]:
# You probably wan't to query the dataset at some point, so call the dataset to return a query builder for it.
dataset_query_builder = dataset()

# Queries are built by chaining calls to the 'variables', 'for_', and 'in_' methods.
# For example, let's build a query to retrieve the male, female, and total populations of each state.
dataset_query_builder.variables("B01001_002E", "B01001_026E", "B01001_001E").for_(("state", "*"))

# Execute the query by calling the builder object
dataset_query_builder()
# NOTE: you can also used the "query()" method (e.g., dataset_query_builder.query())

# The examples above are actually more verbose than necessary;
# you can concisely construct and execute queries like this:
dataset().variables("B01001_002E", "B01001_026E", "B01001_001E").for_(("state", "*"))()

In [ ]:
# In the above examples, you still needed to know the variable codes (e.g., "B06009_001E" for total population) to build the query.
# Alternatively, you can also specify variables using members of the "Variables" enumeration attached to each dataset!
dataset().variables(
    dataset.Variables.Sex_by_Age_Estimate_Total,
    dataset.Variables.Black_or_African_American_Alone_or_in_Combination_With_One_or_More_Other_Races_Estimate_Total
    ).for_((dataset.Geography.county, "*"))()

# NOTE: Some datasets have tens of thousands of variables!
# In these scenarios, autocomplete on the "Variables" enumeration is super helpful (VS Code, for example, also provides a fuzzy search)!
# You can also navigate to the definition of the enumeration in your IDE to see the full list.

In [2]:
from census_products.cbp import Cbp_2022 as cbp

In [14]:
# QUery
df = cbp().variables(
    cbp.Variables.All_Sectors_County_Business_Patterns_including_ZIP_Code_Business_Patterns_by_Legal_Form_of_Organization_and_Employment_Size_Class_for_the_U_S_States_and_Selected_Geographies_2022_Number_of_establishments,
    cbp.Variables.All_Sectors_County_Business_Patterns_including_ZIP_Code_Business_Patterns_by_Legal_Form_of_Organization_and_Employment_Size_Class_for_the_U_S_States_and_Selected_Geographies_2022__017_NAICS_code,
).for_((cbp.Geography.state, "06"))()

# Cast types
df["ESTAB"] = df["ESTAB"].astype("int")

# Sort by NAICS code
df["NAICS Code Length"] = df["NAICS2017"].str.len()
df.sort_values(["NAICS Code Length", "NAICS2017"], inplace=True)
# df.drop(columns=["NAICS Code Length"], inplace=True)

# Map NAICS code to title
xl_map = pd.read_excel(r"C:\Users\btone\Downloads\2-6 digit_2022_Codes.xlsx", dtype="str")
xl_map.drop(columns=["Unnamed: 3", "Unnamed: 4"], inplace=True)
xl_map = xl_map.tail(-1)
new_rows = []
for index, row in xl_map[xl_map["2022 NAICS US   Code"].str.contains("-")].iterrows():
    start_code, end_code = map(int, row["2022 NAICS US   Code"].split("-"))
    for code in range(start_code, end_code + 1):
        new_row = row.copy()
        new_row["2022 NAICS US   Code"] = str(code)
        new_rows.append(new_row.to_frame().T)
x1_map = pd.concat([xl_map.copy()] + new_rows, axis=0, ignore_index=True)

## Convert ranges to rows
naics_code_to_title_map_download = dict(x1_map[x1_map.columns[-2:]].to_records(index=False).tolist())
naics_code_to_title_map_variable = dict(cbp.Variables.All_Sectors_County_Business_Patterns_including_ZIP_Code_Business_Patterns_by_Legal_Form_of_Organization_and_Employment_Size_Class_for_the_U_S_States_and_Selected_Geographies_2022__017_NAICS_code.values[1].value_map)
naics_code_to_title_map = naics_code_to_title_map_download | naics_code_to_title_map_variable
df["2022 NAICS US Title"] = df["NAICS2017"].map(naics_code_to_title_map)

# Reorder
df = df[["state", "NAICS2017", "2022 NAICS US Title", "ESTAB", "NAICS Code Length"]]

In [ ]:
def compute_parent(row):
    naics_code = row["NAICS2017"]
    parent = None

    if naics_code == "00":
        return row
    elif "-" in naics_code:
        # Special handling for ranges
        row["P1"] = "00"
        return row
    
    for i in range(len(naics_code), 1, -1):
        column = f"P{i - 1}"
        row[column] = naics_code[0:i]

    return row

df[["P1", "P2", "P3", "P4", "P5"]] = None
df_with_parents = df.apply(compute_parent, axis=1)
df_with_parents[["P1", "P2", "P3", "P4", "P5"]] = df_with_parents[["P1", "P2", "P3", "P4", "P5"]].replace(naics_code_to_title_map)
# df_with_parents["Parent"] = df_with_parents["Parent"].map(naics_code_to_title_map)

with pd.option_context("display.max_rows", None):
    display(df_with_parents)

In [ ]:
import plotly.express as px

fig = px.sunburst(df_with_parents[df_with_parents["P5"].notna()], path=["P1", "P2", "P3", "P4", "P5"], values="ESTAB")
fig.update_layout(
    autosize=False,
    width=2000,
    height=2000,
)
config = {'scrollZoom': True}
fig.show(config=config)
# fig.write_image("ca_bus.png", scale=20)